In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
from iflow.dataset import gen_cycle_dataset
from data.human_robot_interaction_data.read_hh_hr_data import read_data
from iflow.dataset.iros_dataset import IROS

import torch
import torch.optim as optim
from iflow.dataset import drums_dataset, gen_cycle_dataset
from torch.utils.data import DataLoader
from iflow import model
from iflow.trainers import cycle_dynamics_train
from iflow.utils.generic import to_torch

import matplotlib.pyplot as plt
import numpy as np

from iflow.visualization import visualize_vector_field, visualize_trajectories
from iflow.test_measures.log_likelihood import cycle_log_likelihood

In [ ]:
data_p, data_q, names, times = read_data('data/human_robot_interaction_data/hh/p1/hand_shake_s1_1.csv')
segments = np.load('data/human_robot_interaction_data/hh/segmentation/hand_shake_1.npy')
trajs = [data_p[s[0]:s[1], :, :] for s in segments]

# downsample
stepsize = 1
cutoff = 400

trajs_downsampled = np.array([a[::stepsize,:,:][:cutoff] for a in trajs])
trajs_downsampled.shape
trajs = trajs_downsampled.reshape(19,cutoff,26*3)[:,:,-2:]

In [ ]:
print(trajs.shape)
p = -1
for i in range(0,10):
    plt.plot(-trajs_downsampled[i,:,p,2],trajs_downsampled[i,:,p,1])

In [ ]:
#trajs = np.load('data/DRUMS_dataset/Drums.npy')
#trajs = np.load('data/IROS_dataset/RShape.npy')

In [ ]:
data = gen_cycle_dataset.GENCYCLE(trajs, 2)

In [ ]:
batch_size = 100
depth = 10
## optimization ##
lr = 0.001
weight_decay = 0.1
## training variables ##
nr_epochs = 1000

device = torch.device('cpu')

In [ ]:
def main_layer(dim):
    return  model.ResNetCouplingLayer(dim)

def create_flow_seq(dim, depth):
    chain = []
    for i in range(depth):
        chain.append(main_layer(dim))
        chain.append(model.RandomPermutation(dim))
        chain.append(model.LULinear(dim))
    chain.append(main_layer(dim))
    return model.SequentialFlow(chain)

In [ ]:
dim = data.dim
T_period = (2*np.pi)/data.w
params = {'batch_size': batch_size, 'shuffle': True}
dataloader = DataLoader(data.dataset, **params)

lsd = model.LinearLimitCycle(dim, device, dt=data.dt, T_period=T_period)
flow = create_flow_seq(dim, depth)
iflow = model.ContinuousDynamicFlow(dynamics=lsd, model=flow, dim=dim).to(device)

params = list(flow.parameters()) + list(lsd.parameters())
optimizer = optim.Adamax(params, lr = lr, weight_decay= weight_decay)

In [9]:
for i in range(nr_epochs):
    # Training
    for local_x, local_y in dataloader:
        dataloader.dataset.set_step()
        optimizer.zero_grad()
        loss = cycle_dynamics_train(iflow, local_x, local_y)
        loss.backward(retain_graph=True)
        optimizer.step()

    ## Validation ##
    if i%10 == 0:
        with torch.no_grad():
            iflow.eval()

            # plotting
            out = iflow.generate_trj(torch.from_numpy(data.train_data[0][0][None, :]).float().to(device), data.train_data[0].shape[0])
            out = out.detach().cpu().numpy()
            plt.plot(out[:,0], out[:,1], 'r')
            for t in data.train_data:
                plt.plot(t[:,0], t[:,1])
            plt.show()

            # likelihood
            step = 20
            trj = data.train_data[0]
            trj_x0 = to_torch(trj[:-step,:], device)
            trj_x1 = to_torch(trj[step:,:], device)
            phase = to_torch(data.train_phase_data[0][:-step], device)
            cycle_log_likelihood(trj_x0, trj_x1, phase, step, iflow, device)

KeyboardInterrupt: 

In [ ]:
fig_number = 3
val_trj = data.train_data


_trajs = np.zeros((0, 2))
for trj in val_trj:
    _trajs = np.concatenate((_trajs, trj),0)
min = _trajs.min(0) - 0.5
max = _trajs.max(0) + 0.5

n_sample = 100

x = np.linspace(min[0], max[0], n_sample)
y = np.linspace(min[1], max[1], n_sample)

xy = np.meshgrid(x, y)
h = np.concatenate(xy[0])
v = np.concatenate(xy[1])
hv = torch.Tensor(np.stack([h, v]).T).float()
if device is not None:
    hv = hv.to(device)

hv_t1 = iflow.evolve(hv, T=3)
hv = hv.detach().cpu().numpy()
hv_t1 = hv_t1.detach().cpu().numpy()

vel = (hv_t1 - hv)

vel_x = np.reshape(vel[:, 0], (n_sample, n_sample))
vel_y = np.reshape(vel[:, 1], (n_sample, n_sample))
speed = np.sqrt(vel_x ** 2 + vel_y ** 2)
speed = speed/np.max(speed)

fig = plt.figure(fig_number, figsize=(10, 10))
plt.clf()
ax = plt.gca()

plt.streamplot(xy[0], xy[1], vel_x, vel_y, density=[0.5, 1])
for i in range(len(val_trj)):
    plt.plot(val_trj[i][:,0], val_trj[i][:,1], 'b')
plt.draw()
plt.pause(0.05)
plt.show()